N.b. This requries a local large language model to be running. Its rquired that users download OLLAMA before they can do anything https://github.com/ollama/ollama 

The Ollama server also needs to be running for this code to work

I.e. #ollama run llama3.2

In [ ]:
from itertools import combinations
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import re
import json
import copy
import nltk
from nltk.tokenize import sent_tokenize
from google import genai
import ollama
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
nltk.download("punkt")

In [ ]:
datasets = []
publications = []
tools = []
with open ('./data/datasets.json') as f:
    datasets = json.load(f)
with open ('./data/publications.json') as f:
    publications = json.load(f)
with open ('./data/tools.json') as f:
    tools = json.load(f)

len(datasets), len(publications), len(tools)

In [ ]:
titles = [x['metadata']['summary']['title'] for x in datasets]
titles[:5]

In [ ]:
descriptions = [x['metadata']['summary']['description'] for x in datasets]
descriptions[:5]

In [121]:
paper_abstracts = [x['abstract'] for x in publications]
paper_doi = [x['paper_doi'] for x in publications]
paper_abstracts[1]
abstract = paper_abstracts[50]

print(abstract)

<h4>Background</h4>Clinical trials generally each collect their own data despite routinely collected health data (RCHD) increasing in quality and breadth. Our aim is to quantify UK-based randomised controlled trials (RCTs) accessing RCHD for participant data, characterise how these data are used and thereby recommend how more trials could use RCHD.<h4>Methods</h4>We conducted a systematic review of RCTs accessing RCHD from at least one registry in the UK between 2013 and 2018 for the purposes of informing or supplementing participant data. A list of all registries holding RCHD in the UK was compiled. In cases where registries published release registers, these were searched for RCTs accessing RCHD. Where no release register was available, registries were contacted to request a list of RCTs. For each identified RCT, information was collected from all publicly available sources (release registers, websites, protocol etc.). The search and data extraction were undertaken between January an

In [114]:
def can_you_find_a_dataset(description):
    prompt = (
        "The text below is from a research publication, can you the names of any datasets referenced?"
        "In the response, please give me a comma separated list of names of possible datasets, if you dont think there is one, "
        "dont make up any and return 'none':\n\n" + description
    )
    
    response = ollama.chat(
        model="llama3.2:latest",
        messages=[{"role": "user", "content": prompt}],
    )
    return response

In [115]:

def extract_response(response_text: str) -> str:
     # Ensure that the response_text is a string.
    if not isinstance(response_text, str):
        response_text = str(response_text)
    
    # This regex captures any characters between "content='" and the next "'"
    pattern = r"content='(.*?)'"
    match = re.search(pattern, response_text)
    if match:
        return match.group(1)
    return ""

In [116]:
def check_output(description):
    prompt = (
        "does the following look like a comma seperated list of datasets? give a simple yes or no response" + description
    )
    
    response = ollama.chat(
        model="llama3.2:latest",
        messages=[{"role": "user", "content": prompt}],
    )
    return response


In [ ]:
def get_dataset_names(abstract: str, max_attempts: int = 3) -> str:
    attempt = 0
    valid_datasets = set()

    while attempt < max_attempts:
        # Extract dataset names from the sentence
        response = can_you_find_a_dataset(abstract)
        dataset_names = extract_response(response)
        print(f"Attempt {attempt + 1}")
        print(f"Extracted dataset(s): {dataset_names}")

        # Check if extraction is valid
        check_resp = check_output(dataset_names)
        check_result = extract_response(check_resp)
        print(f"Validation output: {check_result}")

        # Only keep valid dataset names, discard 'none' responses
        if check_result.strip().lower() == "yes":
            valid_datasets.update(dataset_names.split(", "))

    if valid_datasets:
        return ", ".join(valid_datasets)

    attempt += 1

    return ""

dataset_names = get_dataset_names(abstract)
print("names=",dataset_names)

RemoteProtocolError: Server disconnected without sending a response.